### Get data

In [1]:
# nuclio: ignore
import nuclio

In [2]:
# nuclio: start-code

In [3]:
%nuclio config spec.image = "mlrun/ml-models-gpu"
%nuclio config kind = "job"

%nuclio: setting spec.image to 'mlrun/ml-models-gpu'
%nuclio: setting kind to 'job'


In [4]:
from os import path
import pandas as pd

# Ingest a data set into the platform
def get_data(context, data_size):
    data_path = f'/User/nlp/pickle/master_frame_trimmed.pkl'
    print("Loading full dataset...")
    df = pd.read_pickle(data_path)
    print(f"Sampling {data_size} data points...")
    df_sample = df.sample(n=int(str(data_size)), random_state=1)
    
    data_out_path = "/User/nlp/run/data.pkl"
    df_sample.to_pickle(data_out_path)
    context.logger.info(f'Saving data to {data_out_path} ...')
    context.log_result('data', data_out_path)

In [5]:
# nuclio: end-code

In [6]:
from mlrun import mlconf
import os
from os import path

# Target location for storing pipeline artifacts
artifact_path = path.abspath('../jobs')
# MLRun DB path or API service URL
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

print(f'Artifacts path: {artifact_path}\nMLRun DB path: {mlconf.dbpath}')

Artifacts path: /User/nlp/components/jobs
MLRun DB path: http://mlrun-api:8080


In [7]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("get_data")

# add metadata (for templates and reuse)
fn.spec.default_handler = "get_data"
fn.spec.description = "load nlp dataset"
fn.metadata.categories = ["data-source", "ml"]
fn.metadata.labels = {"author": "nschenone"}
fn.export("../yaml/get_data.yaml")

> 2020-08-13 19:00:33,339 [info] function spec saved to path: ../yaml/get_data.yaml
